In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/TestSAWO/")

from Rule import Rule
from RuleSet import RuleSet
from Node import Node
from Architecture import Architecture
from SawoNN import SawoNN
from Member import Member

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
me = 0

In [ ]:
import time
import json
import random
import numpy as np
import tensorflow as tf


url = 'http://simviz.herokuapp.com/simviz'


folder_path = "/content/drive/MyDrive/Colab Notebooks/TestSAWO/"
population_path = folder_path + "population/"
control_path = folder_path + "control/"
dataset_path = folder_path + "datasets/"


rs = RuleSet(folder_path + "grammar.txt")
print(rs)

RULE SET
--------
START := { "a": SIZE , "b": SIZE , "c": SIZE } $ self.gu ( 'io' , CONSTANT , ERROR , ARCHITECTURE_IO ) ~ self.gu ( 'ia' , CONSTANT , ERROR , ARCHITECTURE_IA ) ~ self.gu ( 'ib' , CONSTANT , ERROR , ARCHITECTURE_IB ) ~ self.gu ( 'ic' , CONSTANT , ERROR , ARCHITECTURE_IC ) ~ self.gu ( 'ab' , CONSTANT , ERROR , ARCHITECTURE_AB ) ~ self.gu ( 'ba' , CONSTANT , ERROR , ARCHITECTURE_BA ) ~ self.gu ( 'ac' , CONSTANT , ERROR , ARCHITECTURE_AC ) ~ self.gu ( 'ca' , CONSTANT , ERROR , ARCHITECTURE_CA ) ~ self.gu ( 'bc' , CONSTANT , ERROR , ARCHITECTURE_BC ) ~ self.gu ( 'cb' , CONSTANT , ERROR , ARCHITECTURE_CB ) ~ self.gu ( 'ao' , CONSTANT , ERROR , ARCHITECTURE_AO ) ~ self.gu ( 'bo' , CONSTANT , ERROR , ARCHITECTURE_BO ) ~ self.gu ( 'co' , CONSTANT , ERROR , ARCHITECTURE_CO ) $ ARCHITECTURE
SIZE := 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20
ARCHITECTURE := ['i', 'o'] | ['i', 'a', 'o'] | ['i', 'ia', 'o'] | ['i', 'b', 'o'] | ['i', 'ib', 'o'

In [ ]:

stages_file = open(control_path + 'stages.json')
stages = json.load(stages_file)
stages_file.close()

while(not stages['finished']):

    if(stages['starting']):
        print("starting...")


    elif(stages['initpop']):
        print("initpop...")

        seed = stages['seed'] + me
        print("seed:", seed)
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        init_filename = control_path + "runner_" + str(me) + "_initpop.json"
        init_file = open(init_filename)
        init = json.load(init_file)
        init_file.close()

        if(not init['finished']):

            evalu_filename = control_path + "runner_" + str(me) + "_evalpop.json"
            evalu_file = open(evalu_filename)
            evalu = json.load(evalu_file)
            evalu_file.close()

            which_to_init = init["which_to_init"]
            for i in range(which_to_init):

                mem = Member()
                chromosome = mem.chromosome
                print(chromosome)

                evalu['chromosomes'][i] = chromosome
            
            init['finished'] = True

            evalu_file = open(evalu_filename, 'w')
            json.dump(evalu, evalu_file)
            evalu_file.close()

            init_file = open(init_filename, 'w')
            json.dump(init, init_file)
            init_file.close()

            print("done initpop")


    elif(stages['evaluate']):
        print("evaluate...")



        evalu_filename = control_path + "runner_" + str(me) + "_evalpop.json"
        evalu_file = open(evalu_filename)
        evalu = json.load(evalu_file)
        evalu_file.close()

        seed = stages['seed'] + (me*20) + evalu["gen"]
        print("seed:", seed)
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        if(not evalu['finished']):

            from sklearn.model_selection import train_test_split

            X = np.load(dataset_path + '10_input.npy')
            print(X.shape)
            y = np.load(dataset_path + '10_target.npy')
            print(y.shape)
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=int(random.random()*123456))
            print(X_train.shape)
            print(y_train.shape)
            print(X_test.shape)
            print(y_test.shape)


            which_to_eval = evalu["which_to_eval"]
            for i in range(which_to_eval):

                chromosome = evalu['chromosomes'][i]
                print()
                print(i, ":", chromosome)
                mem = Member(chromosome)

                fit = mem.fitness(rs, X_train, X_test, y_train, y_test)

                evalu['fitnesses'][i] = fit

            
            evalu['finished'] = True

            evalu_file = open(evalu_filename, 'w')
            json.dump(evalu, evalu_file)
            evalu_file.close()

            print("done evaluate")

    elif(stages['select_and_alter']):
        print("select_and_alter...")
    
    time.sleep(random.random() * 5) #seconds

    stages_file = open(control_path + 'stages.json')
    stages = json.load(stages_file)
    stages_file.close()
